In [ ]:
import numpy as np
import roadrunner
import pandas as pd
import matplotlib.pyplot as plt

SEED=1
np.random.seed(seed=SEED)

# Model system parameters
MODEL_SBML_FILE = "model_Michaelis_Menten.xml"
SPECIES = ["S", "E", "ES", "P"]
SPECIES_IDs = ["S1", "S2", "S3", "S4"] # how the species ids are saved in the SBML file from the original BNGL model
DURATION = 20000
PARAMS = ["log_k1", "log_k2", "log_k3"]

# Synthetic dataset parameters
NOISE = 0.01 # % Gaussian noise (range is 0-1)
N_PTS = 11

In [ ]:
def gaussian_noise(x,std,mu=0):
    noise = np.random.normal(mu, std, size = x.shape)
    print(noise)
    x_noisy = x + noise
    return x_noisy

rr = roadrunner.RoadRunner(MODEL_SBML_FILE)
rr.integrator.absolute_tolerance = 5e-10
rr.integrator.relative_tolerance = 1e-8

rr.resetAll()
res = rr.simulate(0, DURATION, N_PTS)
# ignore the first timepoint (i.e. t=0)
ts_data = res[1:, 0]
prod_traj = res[1:, -1]

std = NOISE * prod_traj#np.std(prod_traj)
print("STD:", std)
noisy_data = gaussian_noise(prod_traj, std)
print(noisy_data)

In [ ]:
plt.plot(ts_data, prod_traj, 'r-', lw=0.5, label="Original")
plt.plot(ts_data, noisy_data, 'rx', label="Noisy Data")

In [ ]:
N_DATA = len(ts_data)
# Save for later
df = pd.DataFrame()

df["observableId"] = ["product"] * N_DATA
df["simulationConditionId"] = ["simCondition"] * N_DATA
df["measurement"] = noisy_data
df["time"] = ts_data
#df["noiseParameters"] = [0] * N_DATA
df.to_csv("measurementData_Michaelis_Menten.tsv", sep="\t", index=False)
df

In [ ]:
!petablint -h

In [ ]:
!petablint -y ./Michaelis_Menten.yaml